In [69]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split

from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier


In [70]:
#数据的导入
data_train=pd.read_csv('train.csv')           #读取训练数据
data_test=pd.read_csv('test.csv')             #读取测试数据
Y=data_train.target                           #读取训练数据中的target字段  输出

In [71]:
#对数据进行预处理
data_train.dropna(axis=0,how='any',subset=['target'],inplace=True)         #删除Score为空值的数据行

In [72]:
X=data_train.drop(['target','id'], axis=1)                
X=pd.get_dummies(X)                                                    #用pd.get_dummies方法进行one-hot编码                                  
X=X.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))               #min-max标准化（Min-Max Normalization）
                                                                       #也称为离差标准化，是对原始数据的线性变换，使结果值映射到[0-1]

test_X=data_test.drop(['id'], axis=1)                
test_X=pd.get_dummies(test_X)
test_X=test_X.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))        

X=np.array(X)
X=pd.DataFrame(X)                           #加这两行是为了让dataframe的列名变为按序排列的数字，为什么用原来的列名总会出现使得X^T*X为奇异矩阵
test_X=np.array(test_X)
test_X=pd.DataFrame(test_X)                 
X_total=pd.concat([X,test_X])               #将训练测试集合并起来，统一进行数据处理

In [73]:
cols=[x for i,x in enumerate(X_total.columns) if X_total.iat[0,i]==0]              #得到全为0的列
X_total=X_total.drop(cols,axis=1)                                                   #删掉全为0的列

In [74]:
X=X_total[0:49502]
test_X=X_total[49502:61878]

In [75]:
# 将数据分为训练集和校验集
train_X, valid_X, train_Y, valid_Y = train_test_split(X.values, Y.values, test_size=0.25)

In [76]:
bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=8),
    n_estimators=3000,
    learning_rate=0.01)


#bdt_real.fit(X.values, Y.values)
bdt_real.fit(train_X, train_Y)


AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.01, n_estimators=3000, random_state=None)

In [77]:
Y_pred=bdt_real.predict_proba(test_X)

In [78]:
answer = pd.read_csv(open("./sampleSubmission.csv"))
class_list = ["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"]
answer[class_list] = answer[class_list].astype(float)

# 将答案放进去
j = 0
for class_name in class_list:
    answer[class_name] = Y_pred[:, j]
    j += 1
answer.to_csv("./submission.csv",index=False) # 不要保存引索列